In [1]:
import nest_asyncio
nest_asyncio.apply()

import pandas as pd
from IPython.display import display, HTML
from copy import deepcopy

# Llama Index imports
from llama_index.core import (
    VectorStoreIndex, 
    SimpleDirectoryReader, 
    get_response_synthesizer, 
    QueryBundle
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.prompts import PromptTemplate
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core import Settings

/cherry/projects/rag_llama_index/rag4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuring LLM and Embeddings
Settings.llm = Ollama(model="llama3.1", request_timeout=300.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.chunk_overlap = 0
Settings.chunk_size = 512

In [4]:
# Load documents from PDF
documents = SimpleDirectoryReader(input_files=["ViT.pdf"]).load_data()

# Create VectorStore index
index = VectorStoreIndex.from_documents(documents)

In [5]:
# Set display option for pandas
pd.set_option("display.max_colwidth", 1)

# Helper function for rendering pandas DataFrame in HTML
def pretty_print(df):
    display(HTML(df.to_html().replace("\\n", "")))

In [6]:
# Visualization of retrieved nodes
def visualize_retrieved_nodes(nodes):
    result_dicts = [
        {"Score": node.score, "Text": node.node.get_text().replace("\n", " ")}
        for node in deepcopy(nodes)
    ]
    pretty_print(pd.DataFrame(result_dicts))

In [7]:
# Function to retrieve and answer the query
def get_answer_and_retrieved_nodes(query_str, vector_top_k=10, reranker_top_n=5, use_reranker=False):
    # Bundle the query
    query_bundle = QueryBundle(query_str)

    # Configure retriever
    retriever = VectorIndexRetriever(index=index, similarity_top_k=vector_top_k)

    # Configure response synthesizer
    response_synthesizer = get_response_synthesizer()

    # Define template for generating answers
    template = """
    You are a knowledgeable and precise assistant specialized in question-answering tasks, 
    particularly from academic and research-based sources. 
    Your goal is to provide accurate, concise, and contextually relevant answers based on the given information.

    Instructions:
    - Comprehension and Accuracy: Carefully read and comprehend the provided context from the research paper.
    - Conciseness: Deliver the answer in no more than three sentences.
    - Truthfulness: If the context does not provide enough information, clearly state, "I don't know."
    - Contextual Relevance: Ensure your answer is supported by the context and does not include external information.

    Here is the question and context for you to work with:
    \nQuestion: {question} \nContext: {context} \nAnswer:
    """

    prompt_tmpl = PromptTemplate(
        template=template,
        template_var_mappings={"query_str": "question", "context_str": "context"}
    )

    # Retrieve nodes
    retrieved_nodes = retriever.retrieve(query_bundle)

    # Rerank nodes if reranker is enabled
    if use_reranker:
        reranker = SentenceTransformerRerank(
            model="cross-encoder/ms-marco-MiniLM-L-2-v2", 
            top_n=reranker_top_n
        )
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        reranker = None

    # Assemble the query engine
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer,
        node_postprocessors=[reranker] if reranker else []
    )

    # Update query engine with custom prompt template
    query_engine.update_prompts(
        {"response_synthesizer:text_qa_template": prompt_tmpl}
    )

    # Get synthesized answer
    answer = query_engine.query(query_bundle)

    return retrieved_nodes, answer

In [8]:
query_str = "How do Vision Transformers (ViT) leverage self-attention mechanisms to integrate global information in the early layers of image processing?"

In [9]:
retrieved_nodes, answer = get_answer_and_retrieved_nodes(query_str, 
                use_reranker=True,
                reranker_top_n=3
                  )

print("Answer:\n", answer)
print("Retrieved Nodes:\n")
visualize_retrieved_nodes(retrieved_nodes)

Answer:
 The Vision Transformer (ViT) leverages self-attention mechanisms to integrate global information in the early layers of image processing by applying it on patches extracted from the input image. This allows ViT to attend to most of the image already in the lowest layers, demonstrating that the ability to integrate information globally is indeed used by the model. The attention distance increases with network depth, and the model attends to image regions that are semantically relevant for classiﬁcation.
Retrieved Nodes:



,Score,Text
0,5.800021,"In a different line of work, Sparse Transformers (Child et al., 2019) employ scalable approximations to global self- attention in order to be applicable to images. An alternative way to scale attention is to apply it in blocks of varying sizes (Weissenborn et al., 2019), in the extreme case only along individual axes (Ho et al., 2019; Wang et al., 2020a). Many of these specialized attention architectures demonstrate promising results on computer vision tasks, but require complex engineering to be implemented efﬁciently on hardware accelerators. Most related to ours is the model of Cordonnier et al. (2020), which extracts patches of size 2 ×2 from the input image and applies full self-attention on top. This model is very similar to ViT, but our work goes further to demonstrate that large scale pre-training makes vanilla transformers competitive with (or even better than) state-of-the-art CNNs. Moreover, Cordonnier et al. (2020) use a small patch size of 2 ×2 pixels, which makes the model applicable only to small-resolution images, while we handle medium-resolution images as well. There has also been a lot of interest in combining convolutional neural networks (CNNs) with forms of self-attention, e.g. by augmenting feature maps for image classiﬁcation (Bello et al., 2019) or by further processing the output of a CNN using self-attention, e.g. for object detection (Hu et al., 2018; Carion et al., 2020), video processing (Wang et al., 2018; Sun et al., 2019), image classiﬁcation (Wu et al., 2020), unsupervised object discovery (Locatello et al., 2020), or uniﬁed text-vision tasks (Chen et al., 2020c; Lu et al., 2019; Li et al., 2019). Another recent related model is image GPT (iGPT) (Chen et al., 2020a), which applies Transformers to image pixels after reducing image resolution and color space."
1,5.538682,"The ﬁrst layer of the Vision Transformer linearly projects the ﬂattened patches into a lower-dimensional space (Eq. 1). Figure 7 (left) shows the top prin- cipal components of the the learned embedding ﬁlters. The com- ponents resemble plausible basis functions for a low-dimensional representation of the ﬁne structure within each patch. After the projection, a learned position embedding is added to the patch representations. Figure 7 (center) shows that the model learns to encode distance within the image in the similarity of position em- beddings, i.e. closer patches tend to have more similar position em- beddings. Further, the row-column structure appears; patches in the same row/column have similar embeddings. Finally, a sinusoidal structure is sometimes apparent for larger grids (Appendix D). That the position embeddings learn to represent 2D image topology ex- plains why hand-crafted 2D-aware embedding variants do not yield improvements (Appendix D.4). Self-attention allows ViT to integrate information across the entire image even in the lowest layers. We investigate to what degree the network makes use of this capability. Speciﬁcally, we compute the average distance in image space across which information is integrated, based on the attention weights (Figure 7, right). This “attention distance” is analogous to receptive ﬁeld size in CNNs. We ﬁnd that some heads attend to most of the image already in the lowest layers, showing that the ability to integrate information globally is indeed used by the model. Other attention heads have consistently small attention distances in the low layers. This highly localized attention is less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right), suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the attention distance increases with network depth. Globally, we ﬁnd that the model attends to image regions that are semantically relevant for classiﬁcation (Figure 6). 4.6 S ELF -SUPERVISION Transformers show impressive performance on NLP tasks."
2,3.620768,"Published as a conference paper 

In [10]:
# Without Reranker
retrieved_nodes, answer = get_answer_and_retrieved_nodes(query_str, 
                          use_reranker=False)

print("Answer:\n", answer)
print("Retrieved Nodes:\n")
visualize_retrieved_nodes(retrieved_nodes)

Answer:
 **Rewrite**

Vision Transformers (ViT) leverage self-attention mechanisms to integrate global information in the early layers of image processing by enabling the network to attend to all pixels simultaneously, which is somewhat surprising given the dominance of convolutional architectures for image data. This global integration capability allows ViT to capture long-range dependencies within an image, as demonstrated in Figure 6, where attention from output tokens to input space reveals that some heads can attend to most of the image even at low layers. The use of self-attention mechanisms motivates future scaling efforts, as Vision Transformers appear not to saturate within the range tried, and holds promise for leveraging global information in image processing tasks.
Retrieved Nodes:



,Score,Text
0,0.853568,"In a different line of work, Sparse Transformers (Child et al., 2019) employ scalable approximations to global self- attention in order to be applicable to images. An alternative way to scale attention is to apply it in blocks of varying sizes (Weissenborn et al., 2019), in the extreme case only along individual axes (Ho et al., 2019; Wang et al., 2020a). Many of these specialized attention architectures demonstrate promising results on computer vision tasks, but require complex engineering to be implemented efﬁciently on hardware accelerators. Most related to ours is the model of Cordonnier et al. (2020), which extracts patches of size 2 ×2 from the input image and applies full self-attention on top. This model is very similar to ViT, but our work goes further to demonstrate that large scale pre-training makes vanilla transformers competitive with (or even better than) state-of-the-art CNNs. Moreover, Cordonnier et al. (2020) use a small patch size of 2 ×2 pixels, which makes the model applicable only to small-resolution images, while we handle medium-resolution images as well. There has also been a lot of interest in combining convolutional neural networks (CNNs) with forms of self-attention, e.g. by augmenting feature maps for image classiﬁcation (Bello et al., 2019) or by further processing the output of a CNN using self-attention, e.g. for object detection (Hu et al., 2018; Carion et al., 2020), video processing (Wang et al., 2018; Sun et al., 2019), image classiﬁcation (Wu et al., 2020), unsupervised object discovery (Locatello et al., 2020), or uniﬁed text-vision tasks (Chen et al., 2020c; Lu et al., 2019; Li et al., 2019). Another recent related model is image GPT (iGPT) (Chen et al., 2020a), which applies Transformers to image pixels after reducing image resolution and color space."
1,0.826781,"Published as a conference paper at ICLR 2021 inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well when trained on insufﬁcient amounts of data. However, the picture changes if the models are trained on larger datasets (14M-300M images). We ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When pre-trained on the public ImageNet-21k dataset or the in-house JFT-300M dataset, ViT approaches or beats state of the art on multiple image recognition benchmarks. In particular, the best model reaches the accuracy of 88.55% on ImageNet, 90.72% on ImageNet-ReaL, 94.55% on CIFAR-100, and 77.63% on the VTAB suite of 19 tasks. 2 R ELATED WORK Transformers were proposed by Vaswani et al. (2017) for machine translation, and have since be- come the state of the art method in many NLP tasks. Large Transformer-based models are often pre-trained on large corpora and then ﬁne-tuned for the task at hand: BERT (Devlin et al., 2019) uses a denoising self-supervised pre-training task, while the GPT line of work uses language mod- eling as its pre-training task (Radford et al., 2018; 2019; Brown et al., 2020). Naive application of self-attention to images would require that each pixel attends to every other pixel. With quadratic cost in the number of pixels, this does not scale to realistic input sizes. Thus, to apply Transformers in the context of image processing, several approximations have been tried in the past. Parmar et al. (2018) applied the self-attention only in local neighborhoods for each query pixel instead of globally. Such local multi-head dot-product self attention blocks can completely replace convolutions (Hu et al., 2019; Ramachandran et al., 2019; Zhao et al., 2020)."
2,0.826703,"Thus, Vision Transformer matches or exceeds the state of the art on many image classiﬁcation datasets, whilst being relatively cheap to pre-train. While these initial results are encouraging, many challenges remain. One is to apply ViT to other com

: 